In [3]:
import pandas as pd
from pandasql import sqldf
import pyparsing as pyp
from os import path
import re
import string

# TODO Solve this problem with the relative paths.
DATA_IN = '../../data/clean/Jefferson_County_KY_Street_Intersections.csv'
assert path.exists(DATA_IN)

df = pd.read_csv(DATA_IN, index_col=0)
df.head(5)

,intid,main_street,cross_street,main_suffix,cross_suffix,cross_prefix,main_prefix,longitude,latitude
OBJECTID,,,,,,,,,
1,154647662,REHL,REHL,RD,CT,W,,-85.510444,38.205887
2,254646551,REHL,TUCKER STATION,RD,RD,,,-85.528169,38.200376
3,354646551,REHL,TUCKER STATION,RD,RD,,,-85.528419,38.200360
4,431949996,I 64 EAST,I 265 RAMP,,,,,-85.504954,38.222603
5,593499996,I 265 NORTH,I 265 RAMP,,,,,-85.505546,38.222127


In [4]:
MST = df.main_street
XST = df.cross_street

streets = pd.concat((MST, XST)).unique()

In [5]:

# Some street names start with digits


digit = '[0-9]'

def searcher(pattern):
    def do(value):
        match = re.search(pattern, value)
        return bool(match)
    return do

hasdigit = searcher(digit)
    
NST = {street for street in streets if hasdigit(street)}
NST = pd.DataFrame(list(NST), columns=['name'])

NST['hasdigit'] = NST.name.apply(searcher('[0-9]'))
NST['firstdigit'] = NST.name.apply(searcher('^[0-9]'))
NST

,name,hasdigit,firstdigit
0,KFEC GATE6,True,False
1,33RD,True,True
2,HIGHWAY 329,True,False
3,19TH,True,True
4,13TH,True,True
...,...,...,...
80,36TH,True,True
81,5TH,True,True
82,34TH,True,True
83,IN-265 SOUTH,True,False


In [6]:

#prefixes]
starts = NST.name.str.get(0).unique()
prefixes = [char for char in starts if not str.isnumeric(char)]
match = lambda v:any(v.startswith(prefix) for prefix in prefixes)
matches = pd.DataFrame(NST[NST.name.apply(match)].name)
matches['start'] = NST.name.str.get(0)
display(matches.groupby('start')['name'].unique())


start
H    [HIGHWAY 329, HIGHWAY 42, HIGHWAY 148, HIGHWAY...
I    [IN-265 RAMP, I 264 WEST, I 65 SOUTH, I 71 RAM...
K    [KFEC GATE6, KFEC GATE2, KFEC GATE3, KY 841, K...
S                       [SYLVANIA NO 4, SYLVANIA NO 6]
U                                     [U S HIGHWAY 42]
Name: name, dtype: object

In [7]:
matches.name = matches.name.str.replace('-', ' ')

def get_prefix(value:str):
    first, *junk = value.split()
    return first

matches['prefix'] = matches.name.apply(get_prefix)

matches.groupby('prefix').name.unique()

matches.update(matches[matches.name.str.startswith('U')])
matches['prefix'] = (matches.name.str.replace("U S", "US")).apply(get_prefix)
#prefixes = matches.groupby('prefix').name.unique().index.to_list()

matches.groupby('prefix').name.unique()



prefix
HIGHWAY     [HIGHWAY 329, HIGHWAY 42, HIGHWAY 148, HIGHWAY...
I           [I 264 WEST, I 65 SOUTH, I 71 RAMP, I 265 NORT...
IN                  [IN 265 RAMP, IN 265 NORTH, IN 265 SOUTH]
KFEC         [KFEC GATE6, KFEC GATE2, KFEC GATE3, KFEC GATE4]
KY                                      [KY 841, KY 841 RAMP]
SYLVANIA                       [SYLVANIA NO 4, SYLVANIA NO 6]
US                                           [U S HIGHWAY 42]
Name: name, dtype: object

In [8]:
def name_parse(name):
    if name.startswith("KFEC"):
        prefix, name = name.split()
        suffix=pd.NA
    elif name.startswith("SYLVANIA"):
        prefix, name = name.split(maxsplit=1)
        suffix = pd.NA
    else:
        prefix, *suffix = name.split()
        suffix = ' '.join(suffix)
    return (prefix, name, suffix)


parsed = matches.name.apply(name_parse)

In [9]:

digit = '[0-9]'

def parse_name(name):
    if re.search("[0-9]", name):
        if re.search("^[0-9]", name):
            # Name starts with number
            ...
        # Name has a number
        ...
    else:
        # Regular name
        return 

In [25]:
def matcher(pattern):
    compp = re.compile(pattern)
    def get_string(string):
        match = compp.search(string)
        if match:
            return True
        else:
            return False
    return get_string

numbered_streets = df[df.cross_street.apply(matcher("^[0-9]"))].cross_street

In [94]:
street_expr = re.compile("(^[0-9]+\w+)(.*)")

def parser(string):
    parsed = street_expr.match(string)
    number, coda =  parsed.groups()
    if not coda:
        coda = pd.NA
    else:
        coda = coda.split()
        if coda == ['STREET']:
            coda = pd.NA
        else:
            if coda[0] == "ST":
                coda[0] = 'STREET'
            coda = ' '.join(coda)

    return {'number':number, 'coda':coda}

def do(series):
    out = list()
    for index, value in series.items():
        value['OBJECTID'] = index
        out.append(value)
    return pd.DataFrame.from_records(out, index='OBJECTID')
    


parsed = numbered_streets.apply(parser)
do(parsed)

,number,coda
OBJECTID,,
8265,38TH,<NA>
8271,38TH,<NA>
8280,38TH,<NA>
8326,27TH,<NA>
8346,22ND,<NA>
...,...,...
94289,32ND,<NA>
94590,2ND,<NA>
97788,13TH,<NA>


In [82]:
c = [1,2,3]
c[0]=9
c

[9, 2, 3]

In [ ]:

pp = parsed.apply(lambda x:x.asDict())
for index, dictionary in pp.items():
    dictionary['OBJECTID'] = index
out = pd.DataFrame.from_records(pp.values, index='OBJECTID')
out[out.suffix.notna()].value_counts()